## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os, traceback
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
print(os.environ['CILOGON_REFRESH_TOKEN'])

In [ ]:
slice_name="Slice-comps-only"

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
# Create topology
t = ExperimentTopology()

# Set capacities
cap = Capacities(core=2, ram=6, disk=10)

boot_script = f"sudo yum install -y yum-utils;" \
         f"sudo yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo;" \
         f"sudo yum install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin;" \
         f"sudo systemctl start docker;" \
         f"sudo systemctl enable docker;" \
         f"sudo curl -SL https://github.com/docker/compose/releases/download/v2.5.0/docker-compose-linux-x86_64 -o /usr/local/bin/docker-compose;" \
         f"sudo chmod +x /usr/local/bin/docker-compose;" \


sites = ['RENC']
name="node"
i=0
for s in sites:
    # Add node
    nm = f"{name}-{i}"
    
    n = t.add_node(name=f"{name}-{i}", site=s)
    

    # Set properties
    n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8', boot_script=boot_script)
    
    i += 1

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
    slice_id = reservations[0].slice_id
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
status, slices = slice_manager.slices()
print("Response Status {}".format(status))
if status == Status.OK:
    if len(slices) > 0:
        slice_object=list(filter(lambda s: s.slice_id == slice_id, slices))[0]
        print(slice_object)
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("ReservationId {}".format(s.reservation_id))
        print("Notices       {}".format(s.notices))
        print("State         {}".format(s.reservation_state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        print("Sliver        {}".format(s.sliver))
        print("Lease End Time {}".format(s.lease_end))
        '''
        if s.sliver_type == "NodeSliver":
            for d in s.sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("-------------------------")
                
        print("Notices       {}".format(s.notices))
        print("=====================")
        '''
else:
    print(f"Failure: {slivers}")

In [ ]:
from datetime import datetime, timedelta, timezone
now = datetime.now(timezone.utc)
new_time = now + timedelta(days=2)
status, exception = slice_manager.renew(slice_object=slice_object, new_lease_end_time=new_time.strftime('%Y-%m-%d %H:%M:%S %z'))

if status != Status.OK:
    print(exception)

In [ ]:
status, slices2 = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    if len(slices) > 0:
        slice_object2=list(filter(lambda s: s.slice_id == slice_id, slices2))[0]
        print(slice_object2)
else:
    print(f"Failure: {slices}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))